# 2020-03-23 Workshop: Localization Particle Filter

You have a floor plan of an environment where you are located, but you have no idea where you are on that plan. You have to find out your location based on sensory measurements.

The key advantage of particle filters is that they are really easy to program. In this class you will write your own particle filter for the localization problem.

<img src="https://rse-lab.cs.washington.edu/projects/mcl/animations/global-floor.gif">

In the environment the red dots are particles. They are a discrete guess as to where you might be. These particles are structured as an x coordinate, a y coordinate and also a heading direction (= distance and direction) — three values to comprise a single guess. However, a single guess is not a filter, but rather it is the set of several thousands of guesses that together generate an approximate representation for the posterior of your location.

The essence of particle filters is to have the particles guess where you might be moving, but also have them survive, a kind of "survival of the fittest," so that particles that are more consistent with the measurements, are more likely to survive. As a result, places of high probability will collect more particles, and therefore be more representative of the posterior belief about your location. 


## 1. Particle Filters Theory

### Initial Belief

Generate a large number of particles. Initially the particles are distributed in the area uniformly at random.

<img src="img/ws2/particles_step_1.png" width="600">

### Weights

* Each particle $i$ has a likelihood $p_i$. Each has its own weight: some are very plausible, while others look very implausible as indicated by the size of the particle. The weight of each particle will be $w_i = \frac{p_i}{\sum p_i}$.
* Obtain N new particles from the original set.
    * The resampling is done with replacement.
    * Particles with high weight may have multiple copies.
    * Particles with low weight may disappear.
* New particles will have random destinations according to the mobility model. The higher the weight the more particles will be chosen on that specific position.

<img src="img/ws2/particles_step_2.png" width="600">

We allow the particles to survive at random, but the probability of survival will be proportional to the weights. That is, a particle with a larger weight will survive at a higher proportion than a particle with a small weight. This means that after resampling, which is randomly drawing new particles from the old ones with replacement in proportion to the importance weight, the particles with a higher importance weight will live on, while the smaller ones will die out. The "with replacement" aspect of this selection method is important because it allows us to choose the high probability particles multiple times. This causes the particles to cluster around regions with high posterior probability.

### Resampling

Resampling is the trickiest part of programming a particle filter. Resampling is when you generate a new list of particles by letting some of your old particles survive and killing off others.

Why is it necessary to resample the particles? If you just keep your old particles around forever without resampling them, what happens is that your particles drift around according to your motion  model (transition probabilities for the next time step), but other than their weights, they are unaffected by your observations. Highly unlikely particles  will be kept around and transitioned to more unlikely states, and you might only have say, one particle in the area of high probability of your posterior. So what you end up with is one particle with a way higher likelihood than any of the other particles, and a whole lot of particles with almost-nil probability. This is what we call ‘particle depletion’, because you in effect have only one particle. And one particle doesn’t represent a pdf very well. If you don’t have a lot of particles in the areas of your pdf with high probability, you won’t represent the pdf very well. The density of your particles should be high in high-probability areas, and low in low-probability areas. And so you have to resample the particles, so that they continue to represent the pdf accurately and keep track of many high-probability hypotheses, instead of tracking lots of useless, low-probability hypotheses.

<img src="img/ws2/particles_step_3.png" width="600">

### Movement Model

The accuracy of the motion model is very important!

<img src="img/ws2/particles_step_4.png" width="600">
<img src="img/ws2/particles_step_5.png" width="600">

### Putting it All Together

Back to the math, we need to add one more step: resampling

<img src="img/ws2/particles_step_6.png" width="600">

<div class="alert alert-block alert-success">
<b>Watch and read:</b>
<ul>
    <li>Video: <a href="https://www.youtube.com/watch?v=aUkBa1zMKv4">Particle Filter Explained without Equations</a></li>
    <li>Video: <a href="https://www.youtube.com/watch?v=eAqAFSrTGGY">Short Introduction to Particle Filters and Monte Carlo Localization</a></li>
    <li>Paper: “A Tutorial on Particle Filters for Online Nonlinear/Non-Gaussian Bayesian Tracking”, IEEE Trans. Signal Processing, 2002.</li>
</ul>
</div>


## 2. A Toy Example

Initial belief:
<img src="img/ws2/particles_example_1.png" width="600">

Sense:
<img src="img/ws2/particles_example_2.png" width="600">

We can’t update much. We don’t know where we are, we have no training data, no model, nothing. We can’t calculate weights or do resampling. Let’s skip "sense" and "resampling" states and "move". 

Move, assume you can count your steps.
<img src="img/ws2/particles_example_3.png" width="550">

Resample:
<img src="img/ws2/particles_example_4.png" width="550">

It's important to capture motion accurately.

<img src="img/ws2/particles_example_5.png" width="600">
<img src="img/ws2/particles_example_6.png" width="600">

Particle update in real scenario:

<img src="img/ws2/particles_example_7.png">

<div class="alert alert-block alert-success">
<b>Read:</b>
<ul>
    <li><a href="http://research.microsoft.com/pubs/166309/com273-chintalapudi.pdf">Paper: Zee: Zero-Effort Crowdsourcing for Indoor Localization</a></li>
</ul>
</div>

## 3. Tips and Tricks

Each particle has (x, y, distance, direction). Capturing motion accurately is essential!

### Distance

Use step counting based on standard deviation ($\sigma$) and autocorrelation ($\psi$).
* Method works irrespective of device placement. Tested in pockets, bags, belt, hands.

<img src="img/ws2/distance_stddev.png" width="600">

Acceleration alone is not sufficient, sudden movements such as hand waiving, chair rotation, standing up, also cause acceleration. We need to take advantage of repetitive nature of walking / running and thus compute autocorrelation as well. The period also provides the time taken per step.

Feature: magnitude of acceleration, window = 1 sec. Algorithm and results:
<img src="img/ws2/distance_algorithm.png" width="600">


### Direction 

The phone gives you a relative direction with respect to the cardinal North (like any compass would do). The magnetic offset depends on the building materials and surrounding metallic elements. Very likely this won’t be the most important factor, but we will only test this factor in your app.

<img src="img/ws2/direction.png">

The phone very likely points to a direction different from our moving direction. 

Typical offsets:
* Watching a movie with phone held laterally: 90
* Phone in shirt/pants pockets: +/- 45
* Bags: anywhere

Directionality is not an easy problem. Do it incrementally: Hold the phone in your hand pointing in the same direction of walk.

```
    distance = motion detection + step count + noise
    direction = phone direction + magnetic offset + noise 
```

Try with the phone in your pocket in the next step.


<div class="alert alert-block alert-success">
<b>Read:</b>
<ul>
    <li>Paper: <a href="http://synrg.csl.illinois.edu/papers/walkcompass.pdf">I am a Smartphone and I can Tell my User’s Walking direction”, ACM Mobisys 2014</a></li>
</ul>
</div>


***
# Credits
* Marco Zuniga's <a href="http://studiegids.tudelft.nl/a101_displayCourse.do?course_id=40368">"Smart Phone Sensing" Course at TU Delft</a>
* Parts of this workshop are based on content from: <a href="https://www.udacity.com/course/cs373">Artificial Intelligence for Robotics</a>